# Stylizacja tłumaczeń

<img src="https://live.staticflickr.com/65535/54530431746_73942f964a_n.jpg" alt="Embedded Photo" width="500">

*Obraz wygenerowany przy użyciu modelu DALL-E.*

## Wstęp

Aiga poszła do księgarni i kupiła książkę o deep learningu. Była bardzo podekscytowana — w końcu porządnie odświeży swoją wiedzę przed finałem Olimpiady Sztucznej Inteligencji!

Wróciła do domu, usiadła wygodnie z kubkiem herbaty i zaczęła czytać pierwsze zdania:

> *"Zanurzenia zdaniowe otrzymane z modeli typu enkoder są zwykle lepsze od tylko dekoderowych."*
> 
> 
> *"Uczenie ze wzmocnieniem z ludzkiego nadzoru typowo wykonuje się przez optymalizowanie polityki proksymalnej do modelu nagród."*


— Co...? — mruknęła Aiga, patrząc zdezorientowana na kartki.

Zadzwoniła do kolegi, który od razu jej wyjaśnił:

> *"Sentence embeddingi są lepsze, jeśli wyciągniesz je z encoderów niż z modeli decoder-only."*
> 
> 
> *"RLHF typowo używa PPO, żeby optymalizować politykę względem reward modelu."*


Aiga odetchnęła z ulgą. Teraz wszystko stało się *trochę* jaśniejsze.

Ulga była jednak tylko chwilowa — nadal nie mogła pogodzić się ze stylem tłumaczeń w książce. Postanowiła więc wytrenować własny model tłumaczenia z angielskiego na polski — taki, który będzie tłumaczył terminy związane z AI tak, jak jej się podoba!

## Zadanie

W tym zadaniu Twoim celem jest dostosowanie tłumaczeń generowanych przez istniejący model tłumaczenia maszynowego do specyficznego stylu – zachowania oryginalnego brzmienia niektórych terminów branżowych z dziedziny uczenia maszynowego.

Model bazowy, którego używamy, to [MarianMT](https://huggingface.co/docs/transformers/en/model_doc/marian) – model tłumaczący z języka angielskiego na polski, oparty na architekturze enkoder-dekoder. Domyślnie model ten tłumaczy wszystkie słowa, również terminologię specjalistyczną. Twoim zadaniem będziezaimplementowanie funkcji przetwarzającej dane wejściowe i dotrenowanie tego modelu, aby terminologia specjalistyczna nie była tłumaczona na język polski.

## Dane

Dostępne dla Ciebie w tym zadaniu dane to:

* **Zbiór treningowy** - zawiera angielskie zdania, ich tłumaczenia na język polski oraz listę słów kluczowych, które powinny pozostać nieprzetłumaczone.

* **Zbiór walidacyjny** - służy do oceny skuteczności Twojego podejścia podczas trenowania modelu; ma ten sam format co zbiór treningowy.

Przykłady mają format:

```
{
    "keywords": ["explainable AI"],
    "en": "Developing explainable AI tools is crucial for trust in automated systems.",
    "pl": "Rozwijanie narzędzi explainable AI ma kluczowe znaczenie dla zaufania do zautomatyzowanych systemów.",
}
```

Zbiór testowy, na których finalnie będzie oceniane Twoje rozwiązanie **nie będzie** zawierał polskich tłumaczeń zdań.

## Kryterium Oceny

Twoje rozwiązanie zostanie ocenione na ukrytych danych testowych, za pomocą metryki **BLEU**. Zbiór testowy jest podobny do zbioru walidacyjnego.

BLEU mierzy zgodność *n-gramów* (spójnych ciągów *n* sąsiadujących słów) pomiędzy Twoim tłumaczeniem a pojedynczym zdaniem referencjnym – im większa zgodność, tym wyższy wynik.

## Ograniczenia

* Twoje rozwiązanie będzie testowane na Platformie Konkursowej bez dostępu do internetu oraz w środowisku **z GPU**.
* Trening oraz ewaluacja Twojego rozwiązania na Platformie Konkursowej nie może trwać dłużej niż **10 minut**.
* Lista dopuszczalnych bibliotek: `torch`, `pandas`, `numpy`, `nltk`, `transformers`, `datasets`, `matplotlib`.

## Pliki Zgłoszeniowe

* Ten notebook uzupełniony o Twoje rozwiązanie.

## Ewaluacja

Pamiętaj, że podczas sprawdzania flaga `FINAL_EVALUATION_MODE` zostanie ustawiona na `True`.

Za to zadanie możesz zdobyć pomiędzy 0 a 100 punktów. Jeśli Twój model osiągnie metykę BLEU niższą niż **0.82** na (tajnym) zbiorze testowym na Platformie Konkursowej, Twoje rozwiązanie otrzyma 0 punktów. Jeśli osiągnie wynik wyższy niż **0.86**, zdobędziesz maksymalną liczbę punktów. Pomiędzy tymi progami, Twój wynik będzie skalował się liniowo i zostanie zaokrąglony do liczby całkowitej.

$$
\mathrm{punkty} = 
\begin{cases}
100 & \text{jeśli } \mathrm{BLEU} > 0.86 \\
0 & \text{jeśli } \mathrm{BLEU} < 0.82 \\
100 \cdot \dfrac{\mathrm{BLEU} - 0.82}{0.86 - 0.82} & \text{w przeciwnym razie}
\end{cases}
$$

## Kod Startowy
W tej sekcji inicjalizujemy środowisko poprzez zaimportowanie potrzebnych bibliotek i funkcji. Przygotowany kod ułatwi Tobie efektywne operowanie na danych i budowanie właściwego rozwiązania.

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI PODCZAS WYSYŁANIA ##########################

FINAL_EVALUATION_MODE = False  # W czasie sprawdzania Twojego rozwiązania, zmienimy tą wartość na True

import json
import os
import torch

import matplotlib.pyplot as plt
import numpy as np
from datasets import Dataset
from nltk.translate.bleu_score import sentence_bleu
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI PODCZAS WYSYŁANIA ##########################

from transformers import set_seed
import random

seed = 42

os.environ["PYTHONHASHSEED"] = str(seed)

random.seed(seed)
np.random.seed(seed)

torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

set_seed(seed)

## Ładowanie Danych
W tej części zadania wczytamy dane treningowe.

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################

def load_dataset(json_path: str) -> Dataset:
    with open(json_path, "r", encoding="utf-8") as f:
        lines = f.readlines()

    dataset = []
    for line in lines:
        item = json.loads(line)
        dataset.append(
            {
                "en": item["translation"]["en"],
                "pl": item["translation"]["pl"],
                "keywords": ",".join(item.get("keywords", []))
            }
        )

    return Dataset.from_list(dataset)
    

train_dataset = load_dataset("train_dataset.jsonl")
print(f"Loaded {len(train_dataset)} training examples.")

val_dataset = load_dataset("valid_dataset.jsonl")
print(f"Loaded {len(val_dataset)} validation examples.")

## Kod z kryterium oceniającym

Kod zbliżony do poniższego, będzie używany do oceny rozwiązania na zbiorze testowym.

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################

def evaluate_model(model, tokenizer, dataset, process_example_fn, batch_size=64, device="cuda", verbose=True):
    """
    Funkcja ocenia model na podstawie metryki BLEU.
    Argumenty:
        model: Model do oceny.
        tokenizer: Tokenizer używany do przetwarzania tekstu.
        dataset: Zbiór danych do oceny.
        process_example_fn: Funkcja przetwarzająca przykłady: 
            (en_sentence: str, keywords: List[str]) -> (input_sentence: str).
        verbose: Czy wyświetlać szczegóły dla pierwszych kilku przykładów.
    Wynik:
        float: Średni wynik BLEU dla zbioru danych.
    """
    model.eval()
    model.to(device)

    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size)

    bleu_scores = []

    for batch_idx, batch in enumerate(dataloader):
        orig_sentences = batch["en"]
        reference_translations = batch["pl"]
        keywords = batch["keywords"]

        model_inputs = [
            process_example_fn(orig, kws) 
            for orig, kws in zip(orig_sentences, keywords)
        ]

        inputs = tokenizer(model_inputs, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)

        with torch.no_grad():
            outputs = model.generate(**inputs, max_new_tokens=64)

        hypotheses = tokenizer.batch_decode(outputs, skip_special_tokens=True)

        for i, (ref, hyp) in enumerate(zip(reference_translations, hypotheses)):
            score = sentence_bleu([ref], hyp)
            bleu_scores.append(score)

            if verbose and batch_idx * batch_size + i < 5:
                print(f"Example {batch_idx * batch_size + i}:")
                print(f"Original: {orig_sentences[i]}")
                print(f"Processed: {model_inputs[i]}")
                print(f"Reference: {ref}")
                print(f"Hypothesis: {hyp}")
                print(f"BLEU score: {score:.4f}")
                print("-" * 10)

    bleu_score = sum(bleu_scores) / len(bleu_scores)
    return bleu_score


def compute_score(bleu_score: float) -> float:
    """
    Oblicza wynik punktowy na podstawie wartości metryki BLEU.
    """
    lower_bound = 0.82
    upper_bound = 0.86

    if bleu_score <= lower_bound:
        return 0
    elif lower_bound < bleu_score < upper_bound:
        return int(round(100 * (bleu_score - lower_bound) / (upper_bound - lower_bound)))
    else:
        return 100

## Twoje Rozwiązanie


W tej sekcji należy zaimplementować funkcję `process_example`, wytrenować model i zapisać go jako zmienną o nazwie `my_model`.

In [ ]:
def process_example(en: str, keywords: str) -> str:
    """
    Funkcja, która przetwarza przykłady ewaluacyjne na tekst
    wejściowy do modelu.
    W czasie treningu możesz, ale nie musisz korzystać z tej funkcji.
    Argumenty:
        en: Tekst w języku angielskim.
        keywords: Słowa kluczowe oddzielone przecinkami.
    """
    # TODO: Zaimplementuj funkcję

    return en

In [ ]:
# TODO: Trening modelu.
# Nie wolno zmieniać tokenizatora.
# Pamiętaj, że masz do dyspozycji model oraz tokenizator "gsarti/opus-mt-tc-en-pl".

tokenizer = AutoTokenizer.from_pretrained("gsarti/opus-mt-tc-en-pl")
model = AutoModelForSeq2SeqLM.from_pretrained("gsarti/opus-mt-tc-en-pl")

# TODO: ...

my_model = model  # TODO: Na zmienną "my_model" musisz przypisać swój finalny model.

## Ewaluacja

Uruchomienie poniższej komórki pozwoli sprawdzić, ile punktów zdobyłoby Twoje rozwiązanie na danych walidacyjnych. Przed wysłaniem upewnij się, że cały notebook wykonuje się od początku do końca bez błędów po ustawieniu flagi *FINAL_EVALUATION_MODE = True* i bez konieczności ingerencji użytkownika po wybraniu opcji "Run All".

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################

if not FINAL_EVALUATION_MODE:
    tokenizer = AutoTokenizer.from_pretrained("gsarti/opus-mt-tc-en-pl")
    bleu_score = evaluate_model(
        model=my_model, 
        tokenizer=tokenizer, 
        dataset=val_dataset,
        process_example_fn=process_example,
    )
    print(f"Wynik BLEU zbiorze walidacyjnym: {bleu_score:.4f}")

    score = compute_score(bleu_score)
    print(f"Punkty na zbiorze walidacyjnym: {score}")

Podczas sprawdzania model zostanie zapisany do pliku `your_model.pkl`, a funkcja przetwarzająca do pliku `your_function.pkl` i zostaną one ocenione na zbiorze testowym.

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################

if FINAL_EVALUATION_MODE:
    import cloudpickle

    OUTPUT_PATH = "file_output"

    FUNCTION_OUTPUT_PATH = os.path.join(OUTPUT_PATH, "your_function.pkl")
    MODEL_OUTPUT_PATH = os.path.join(OUTPUT_PATH, "your_model.pkl")

    if not os.path.exists(OUTPUT_PATH):
        os.makedirs(OUTPUT_PATH)

    with open(FUNCTION_OUTPUT_PATH, "wb") as f:
        cloudpickle.dump(process_example, f)

    with open(MODEL_OUTPUT_PATH, "wb") as f:
        cloudpickle.dump(my_model, f)